In [15]:
import warnings
warnings.filterwarnings('ignore')

import os, glob, random, copy
from pathlib import Path
from scipy import stats

import numpy as np
from scipy.stats import rankdata, ttest_rel, ttest_1samp, zscore
from numpy import linalg as LA

from matplotlib import colors
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from matplotlib.patches import Rectangle

import pandas as pd
import seaborn as sns

import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
from nilearn import plotting

from nltools.data import Brain_Data, Adjacency
from nltools.mask import roi_to_brain, expand_mask
from nltools.stats import fdr, threshold, _calc_pvalue
from nltools.utils import get_anatomical

from sklearn.metrics import pairwise_distances

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

import datalad.api as dl

%matplotlib inline

# 1. Establish memory sentiment using sentiment analysis

In [13]:
# set as: 
# 'baseline' for baseline rest
# 'soc' for video encoding, and
# 'rest' for consolidation
stim = 'soc'
exclude_subs = [1,6,14,18,21,31]


# get participant recall data
txt_df = pd.read_csv('./social_memory_freewrites.csv')
# exclude subs
txt_df = txt_df[~txt_df['subject'].isin(exclude_subs)].reset_index(drop=True)
# sort by sub
txt_df = txt_df.sort_values(by=['subject']).reset_index(drop=True)


# COMBINING TEXT ACROSS THE 4 (social or science, depending on 'stim' variable) VIDEOS

subs_txt = []
for sub in range(len(txt_df)):    

    sub_txt=''
    for patient in range (2,6): # looping through columns of the 4 videos
        sub_txt = sub_txt + txt_df[txt_df.columns[patient]].iloc[sub] + '\n'
    subs_txt.append(sub_txt)

# append column which has recall text for all 4 videos
txt_df['soc_txt']=subs_txt
# only retain subID and combined text columns
txt_df = txt_df[['subject','soc_txt']]

# print(txt_df)


# CLEAN TEXT

clean_df = txt_df.copy()
# removing everything except alphabets
clean_df['soc_txt'] = clean_df['soc_txt'].str.replace("[^a-zA-Z#]", " ")
# removing short words
clean_df['soc_txt'] = clean_df['soc_txt'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
# make all text lowercase
clean_df['soc_txt'] = clean_df['soc_txt'].apply(lambda x: x.lower())


from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# tokenization: store each word of corpus as item in list
tokens = clean_df['soc_txt'].apply(lambda x: x.split())
# remove stop-word items
tokens = tokens.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization: recombine list of stop-word excluded words into string corpus
detokenized = []
for sub in range(len(clean_df)):
    token = ' '.join(tokens[sub])
    detokenized.append(token)
    
clean_df['soc_txt'] = detokenized

# Conduct sentiment analysis

In [18]:
def sentiment(txt):
    
    # create list of tokens (words) from string
    tokens = nltk.word_tokenize(txt)
    length = len(tokens)
    
    # 'score' is our main variable of interest, ignore all other variables in code below
    score = abs_score = n_val = 0
    # list of pos & neg words, stored alongwith their scores, for e.g.:
    # ['angry: -.437', 'delighted: .619'...]
    pos_words = neg_words = []
    
    for token in tokens:
        
        # get sentiment score of each word (token)
        token_score = sia.polarity_scores(token)['compound']        
        score = score + token_score
        abs_score = abs_score + np.abs(token_score)
        
        if token_score > 0:
            pos_words.append(f'{token}:{token_score}')
            n_val = n_val + 1
            
        elif token_score < 0:
            neg_words.append(f'{token}:{token_score}')
            n_val = n_val + 1
            
    return pos_words, neg_words, score, abs_score, np.float64(n_val), np.float64(length) 

sub_sent = pd.DataFrame(columns = ['pos_words','neg_words','sentiment_score','abs_score','n_val','length'])

for sub in range(len(clean_df)):
    pos_words, neg_words, score, abs_score, n_val, length = sentiment(clean_df['soc_txt'].iloc[sub])
    sub_sent = sub_sent.append(pd.DataFrame({"pos_words":[pos_words],\
                                             "neg_words":[neg_words],\
                                             "sentiment_score": [score],\
                                             "abs_score": [abs_score],\
                                             "n_val": [n_val],\
                                             "length": [length]},\
                                              index=[sub]))

# store scores in df
soc_sent_df = txt_df.copy()
soc_sent_df['pos_words'] = sub_sent['pos_words']
soc_sent_df['neg_words'] = sub_sent['neg_words']
soc_sent_df['sentiment_score'] = sub_sent['sentiment_score']
soc_sent_df['abs_score'] = sub_sent['abs_score']
soc_sent_df['n_val'] = sub_sent['n_val']
soc_sent_df['length'] = sub_sent['length']

# store scores in df
# the only columns relevant to our interests are:
# 'sentiment_score', 'pos_words', 'neg_words'
soc_sent_df.to_csv(f'./{stim}_sent_all_txt.csv', index=False)


soc_sent_df = pd.read_csv(f'./{stim}_sent_all_txt.csv')
# exclude subs
soc_sent_df = soc_sent_df[~soc_sent_df['subject'].isin(exclude_subs)].reset_index(drop=True)

# a way to store words in df columns in a sorted order based on sentiment scores, i.e.,
# the 'pos_words' column has the most +ve words listed first, and
# the 'neg_words' column has the most -ve words listed first
def top_words(words):
    
    # remove non-words
    words = [w for w in words if w.isalnum() or w in [',', '.', '-', ':', '(', ')']]
    # convert list of words to string
    words = "".join(words)
    # convert string to list (not sure why I'm doing this but I don't want to change the code now)
    words = list(words.split(","))

    dict_words = {}
    
#     print(words)
    
    for word in words:
        # convert list of word-score combos to dict, such that 
        # keys are sentiment scores, and values are the words
        # e.g. converting ['angry: -.437', 'delighted: .619'...] to { -.437: 'angry', .619: 'delighted', ...}
        dict_words[np.abs(float(word.split(':')[1]))] = word.split(':')[0].strip()
    
    # sort dict by score
    dict_words = dict(sorted(dict_words.items()))
    
    return list(dict_words.values())

for sub in range(len(soc_sent_df)):
    soc_sent_df.at[sub,'pos_words'] = list(reversed(top_words(soc_sent_df['pos_words'].loc[sub])))
    soc_sent_df.at[sub,'neg_words'] = top_words(soc_sent_df['neg_words'].loc[sub])

    
soc_sent_df[['pos_words','neg_words','sentiment_score']].head()
soc_sent_df.to_csv(f'./{stim}_sent_all_txt.csv', index=False)


# Convert preprocessed-to-csv fMRI data to (subject x TR x ROI) numpy file

In [19]:
for phase in ['baseline', 'soc_consol', 'sci_consol']:
    
    # grab all files
    files = glob.glob(f'./brain_data/*/*{phase}.csv')
    files.sort()
    data = []

    for file in files:
        data.append(pd.read_csv(file))

    data = np.array(data)
    print('n_subs, n_timepoints, n_parcels')
    print(data.shape)
    # save as subject by TR by ROI numpy file  
    np.save(f'./brain_data/{phase}_nodewise_timeseries', data)


for phase in ['soc_enc', 'sci_enc']:
    
    files = glob.glob(f'./brain_data/*/*{phase}*.csv')
    files.sort()
    data = []
    sub_data = []
    
    for i in range(len(files)):
        
        f = files[i]
        ts = pd.read_csv(f)              

        # trim start and end fixation based on video
        if 'jacob' in f:
            ts = ts.iloc[14:len(ts)-12].reset_index(drop=True)
        elif 'laura' in f:
            ts = ts.iloc[16:len(ts)-14].reset_index(drop=True)
        elif 'molly' in f:                
            ts = ts.iloc[16:len(ts)-21].reset_index(drop=True)
        elif 'morgan' in f:
            ts = ts.iloc[12:len(ts)-12].reset_index(drop=True)
        elif 'path' in f:
            ts = ts.iloc[16:len(ts)-14].reset_index(drop=True)
        elif 'symp' in f:
            ts = ts.iloc[18:len(ts)-9].reset_index(drop=True)
        elif 'diag' in f:                
            ts = ts.iloc[14:len(ts)-18].reset_index(drop=True)
        elif 'gene' in f:
            ts = ts.iloc[12:len(ts)-16].reset_index(drop=True)
        
        # append these 4 (social or science) videos to each subject's data
        sub_data.append(ts)
        
        # if we have traversed through all 4 videos per subject,
        # append this subject's data to the list of all subjects' data, and
        # flush out sub_data variable
        if (i+1)%4 == 0:
            sub_data = pd.concat(sub_data)
            data.append(sub_data.values)
            sub_data = []  

    data = np.array(data)
    print('n_subs, n_timepoints, n_parcels')
    print(data.shape)
    # save as subject by TR by ROI numpy file  
    np.save(f'./brain_data/{phase}_nodewise_timeseries', data)


n_subs, n_timepoints, n_parcels
(40, 360, 50)
n_subs, n_timepoints, n_parcels
(40, 360, 50)
n_subs, n_timepoints, n_parcels
(40, 360, 50)
n_subs, n_timepoints, n_parcels
(40, 1023, 50)
n_subs, n_timepoints, n_parcels
(40, 1023, 50)


# Set up functions & variables for IS-RSA

In [21]:
cmap = plt.cm.get_cmap('RdYlBu_r')
cmap.set_bad('#C0C0C0')

def sort_square_mtx(mtx, vct):
    """
    Sorts rows/columns of a matrix according to a separate vector.
    """
    inds = vct.argsort()
    mtx_sorted = mtx.copy()
    mtx_sorted = mtx_sorted[inds, :]
    mtx_sorted = mtx_sorted[:, inds]    
    return mtx_sorted

def norm_mtx(mtx):
    """
    Scales a matrix to have values between 0 and 1.
    """
#     return mtx/abs(mtx).max()
    return (mtx-np.min(mtx))/(np.max(mtx)-np.min(mtx))


# set up subjects info
exclude_subs = [1,6,14,18,21,31]
n_subs = 46 - len(exclude_subs)

# formatting to print axes of sub-by-sub matrix as 's-1, s-2....X,... s-40'
sub_ls = ['.']*40
sub_ls[0] = 's-1'
sub_ls[1] = sub_ls[3] = ''
sub_ls[2] = 's-2'
sub_ls[4] = 's-3'
sub_ls[39] = 's-40'
sub_y_ls = copy.deepcopy(sub_ls)
sub_ls[16] = 'X'
sub_y_ls[22] = 'Y'


# set up mask parcels info
mask = Brain_Data('http://neurovault.org/media/images/2099/Neurosynth%20Parcellation_0.nii.gz')
masker = NiftiLabelsMasker(labels_img=mask.to_nifti(), standardize=True)


def set_self_rois():
    
    global roi_ntwrk, roi_i, roi_names, n_rois
    
    roi_ntwrk = 'pc-vmpfc'
    roi_i = [6,32]
    roi_names = ['pc', 'vmpfc']
    n_rois = len(roi_i)


def set_dmn_rois():
    
    global roi_ntwrk, roi_i, roi_names, n_rois
    
    roi_ntwrk = 'DMN'
    # indices of relevant rois
    roi_i = [2,5,6,19,32,49]
    roi_names = ['dmpfc', 'tpj', 'pc', 'pcc', 'vmpfc', 'sts']
    n_rois = len(roi_i)
    

def set_dmn_hippo_rois():
    
    global roi_ntwrk, roi_i, roi_names, n_rois
    
    roi_ntwrk = 'DMN_hippo'
    # indices of relevant rois
    roi_i = [2,5,6,19,28,32,49]
    roi_names = ['dmpfc', 'tpj', 'pc', 'pcc', 'hippo', 'vmpfc', 'sts']
    n_rois = len(roi_i)
    
    
def set_limbic_rois():
    
    global roi_ntwrk, roi_i, roi_names, n_rois

    roi_ntwrk = 'limbic'
    # indices of relevant rois
    roi_i = [12,18,22]
    roi_names = ['amyg', 'ai', 'dacc']
    n_rois = len(roi_i)

    
def set_limbic_nacc_rois():
    
    global roi_ntwrk, roi_i, roi_names, n_rois

    roi_ntwrk = 'limbic_nacc'
    # indices of relevant rois
    roi_i = [12,18,22,34]
    roi_names = ['amyg', 'ai', 'dacc', 'nacc']
    n_rois = len(roi_i)
    
    
def set_whole_brain_rois():

    global roi_ntwrk, roi_i, roi_names, n_rois

    roi_ntwrk = 'whole-brain'
    roi_i = list(range(0,50))
    roi_names = pd.read_csv(f'{path}/other_data/neurosynth_chang_parcellation_labels.csv')['label']
    n_rois = len(roi_i)

    # some formatting to name to make roi names cleaner
    for i in range(n_rois):
        roi_names[i] = roi_names[i].replace(" ", "_")
        roi_names[i] = roi_names[i].replace("/", "_OR_")

# store rois/parcels
parcels = pd.read_csv('./neurosynth_chang_parcellation_labels.csv')['label']

# some formatting to name to make roi names cleaner
for i in range(len(parcels)):
    parcels[i] = parcels[i].replace(" ", "_")
    parcels[i] = parcels[i].replace("/", "_OR_")
    
temporal_analysis = False

# 2. Creating subject-pair 1 - mean sentiment matrix (model)

In [ ]:
def sentiment_isc():
    """
    makes & displays sentimentioral annak matrix
    """

    sentiment_df = pd.read_csv('./soc_sent_all_txt.csv')    

# exclude subs
    sentiment_df = sentiment_df[~sentiment_df['subject'].isin(exclude_subs)].reset_index(drop=True)

# only select specific 'score' column
    sentiment_scores = sentiment_df['sentiment_score'].to_numpy()
# convert scores to ranks for spearman correlations
    sentiment_rank = rankdata(sentiment_scores)-1

# append ranks to df
    sentiment_df['score_rank'] = pd.DataFrame(sentiment_rank)


# create sentiment mtx
    sentiment_mtx = np.zeros((n_subs, n_subs))
    for i in range(n_subs):
        for j in range(n_subs):
            if i!=j:
                sim_ij = - 1/n_subs * np.mean([sentiment_rank[i], sentiment_rank[j]])
                sentiment_mtx[i,j] = sentiment_mtx[j,i] =  sim_ij

    adj_sentiment_mtx = Adjacency(sentiment_mtx, matrix_type='similarity')

# plot similarity mtx
    fig = plt.figure(figsize=(30,25))
    
    ax = sns.heatmap(sort_square_mtx(norm_mtx(sentiment_mtx), sentiment_scores), square=True,\
                     xticklabels=sub_ls, yticklabels=sub_y_ls, mask=np.triu(np.ones_like(sentiment_mtx)),\
                     cmap='YlOrRd', vmin=0, vmax=1, cbar=False)

    plt.xlabel('subjects sorted,\nnegative to positive memory affect', fontname='Arial', fontsize=35)
    
    # add a grey border to each matrix cell
    for i in range(n_subs):
        for j in range(n_subs):
            if j>i:
                ax.add_patch(Rectangle((i, j), 1, 1, fill=False, edgecolor='grey', lw=.1))

    ax.set_facecolor("white")
    plt.xticks(fontsize=30, rotation=45, fontname='Arial')
    plt.yticks(fontsize=30, rotation=45, fontname='Arial')
    plt.savefig('./figs/model.png', facecolor='white', edgecolor='none')
    
    return sentiment_mtx, adj_sentiment_mtx, sentiment_df, sentiment_scores

# behavioral isc

print(f'subs = {n_subs}')
sentiment_mtx, adj_sentiment_mtx, sentiment_df, sentiment_scores = sentiment_isc()
subs_sorted = sentiment_df.sort_values('score_rank').index.tolist()


# 3. Creating subject-pair connectivity similarity matrix (data) and performing ISRSA

In [ ]:
def fc():

    global n_subs, phase, roi_ntwrk, roi_i, roi_names, n_rois, start_TR, stop_TR, temporal_analysis

# load neural time series & plot settings 
    ts = np.load(f'./brain_data/{phase}_nodewise_timeseries.npy')
    
# chopping off first 10s and selecting desired rois        
    ts = ts[:, 10: , roi_i]
    total_TR_len = len(ts[2])

# since we dont run temporal analyses on any run besides 'soc_consol',
# the start & end TR (and hence the time window of analysis) is always the entire time series 
    if not phase == 'soc_consol':
        start_TR = 0
        stop_TR = total_TR_len
    
    
# subsetting timeseries window
    ts = ts[:, start_TR : stop_TR, :]
    n_subs, n_tr, n_rois = np.shape(ts)

    
    # list of subjects' connectivity matrices
    sub_mtx_fc = [] # fc mtx

    for sub in range(n_subs):
        
        # create connectivity matrix for each subject
        roi_corr = 1 - pairwise_distances(ts[sub,:,np.arange(n_rois)], metric='correlation')
        
        # append matrix to list of subjects' matrices
        sub_mtx_fc.append(Adjacency(roi_corr, matrix_type='similarity'))
        
# generate subj-by-subj connectivity similarity matrix
    neural_mtx = np.ones((n_subs,n_subs))

    for i in range(n_subs):
        for j in range(n_subs):

            if i!=j:

    # euclidean similarity of subjects' matrices
                    neural_mtx[i,j] = -np.linalg.norm(stats.zscore(sub_mtx_fc[i].squareform()[np.triu_indices(n_rois, k = 1)]) -\
                                                      stats.zscore(sub_mtx_fc[j].squareform()[np.triu_indices(n_rois, k = 1)]))

    # convert matrix from type ndarray to Adjacency
    adj_neural_mtx = Adjacency(neural_mtx, matrix_type='similarity')

    # comute connectivity similarity
    fc_isc = adj_neural_mtx.isc(metric='median', n_bootstraps=4000, n_jobs=-1)    
    
    # compute rsa between model and data (isrsa)
    fc_isrsa = adj_neural_mtx.similarity(adj_sentiment_mtx, metric='spearman', n_permute=4000, n_jobs=-1)
    isrsa_r = np.round(fc_isrsa['correlation'],3)
    isrsa_p = np.round(fc_isrsa['p'],3)              
     
    if not temporal_analysis:
        # plot isrsa
        fig = plt.figure(figsize=(30,25))
        ax = sns.heatmap(sort_square_mtx(norm_mtx(neural_mtx), sentiment_scores), square=True,\
                    xticklabels=sub_ls, yticklabels=sub_y_ls, mask=np.triu(np.ones_like(neural_mtx)),\
                    cmap='YlOrRd', cbar=False)

        ax.set_facecolor("white")
        plt.title(f'Anna Karenina Model Fit for\nDefault Network Connectivity\n{isrsa_r}, p: {isrsa_p}\n', fontsize=35, fontname='Arial')
        plt.xlabel('subjects sorted,\nnegative to positive memory affect', fontsize=35)
        plt.xticks(rotation=45, fontsize=30)
        plt.yticks(rotation=45, fontsize=30)
        plt.savefig(f'./figs/{phase}.png', facecolor='white', edgecolor='none')

    return isrsa_r, isrsa_p, sub_mtx_fc, adj_neural_mtx


# set neural network
# set_whole_brain_rois()
# set_limbic_nacc_rois()
# set_limbic_rois()
set_dmn_rois()
   
temporal_analysis = False

# 1
phase = 'soc_consol' # post-encoding rest
fc()

# 2
phase = 'soc_enc' # video-watching/encoding
fc()

# 3
phase = 'baseline'
fc()


# Temporal Immediacy

In [ ]:
temporal_analysis = True
phase = 'soc_consol'
isrsa_stats = pd.DataFrame(columns=['r', 'p', 'mean_tr', 'tr_window'])

# window_len = the no. of TRs over which we perform analysis
for window_len in [180]:

    # sliding our window each time by 35 TRs
    for start_TR in range(0,180,1):

        print(start_TR, end='')
        stop_TR = start_TR + window_len
        # if the last TR of the window exceeds time series length
        if stop_TR > 350:
            break
        
        r, p, sub_mtx_fc, adj_neural_mtx = fc()
        isrsa_stats.loc[len(isrsa_stats.index)] = [r, p, .5 * (start_TR + stop_TR), f'TR {start_TR}-{stop_TR}']
        
fig = plt.figure(figsize=(10,7))
plt.scatter(isrsa_stats['mean_tr'], isrsa_stats['r'], [5]*len(isrsa_stats))
plt.title('Temporal immediacy of\nAffect Consolidation by the\nDefault Network\n', fontsize=35)
plt.xlabel('TR of social-consolidation scan', fontsize=35, fontname='Arial')
plt.ylabel('Anna Karenina model fit\n(r value)', fontsize=35, fontname='Arial')
plt.xticks(range(0,350,30), rotation = 45, fontsize = 25, fontname='Arial')
plt.yticks(fontsize = 25)

# r_trhesh = y intercept where p = .05
r_thresh=0
for i in range(len(isrsa_stats)):
    
    if isrsa_stats['p'].iloc[i] < .05:        
        r_thresh = isrsa_stats['r'].iloc[i] 

plt.axhline(y = r_thresh, color = 'red', label='p < .05')
plt.legend(loc='upper right', prop={'size': 25})
plt.show()
fig.savefig('./figs/temporal_isrsa.png', facecolor="white", edgecolor='none')


In [ ]:
annak_r = {}
# list of color values for different ROIs
tst = [-100,-75,-50,50,75,100]
i = 0 # index of color list
set_dmn_rois()

for node in range(len(expand_mask(mask))):
    if node not in [2,5,6,19,32,49]:
        annak_r[node] = 0
    else: # only give values to dmn ROIs
        annak_r[node] = tst[i]
        i = i + 1

annak_r_brain = roi_to_brain(pd.Series(annak_r), expand_mask(mask))
cmap = colors.LinearSegmentedColormap.from_list("", ['red','orange','yellow','white','green','cyan','blue'])
plotting.view_img_on_surf(annak_r_brain.to_nifti(), cmap=cmap, black_bg=True)#.save_as_html(f'{figs_path}/brains/brain.html')
